In [ ]:
from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-codes'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 取得優先順位: 展開済み code/ があればそれを使う。無ければ code.zip を展開。どちらも無ければエラー。
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code ディレクトリも code.zip も見つかりません。Add Data で {DATASET_SLUG} を追加してください。")

if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
# unzip でサブディレクトリ1階層挟まった場合に平坦化
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children)==1 and children[0].is_dir():
        inner = children[0]
        for path in inner.iterdir():
            path.rename(repo_dir / path.name)
        inner.rmdir()
%cd $repo_dir
# 依存パッケージは Kaggle 環境の標準ライブラリを利用（追加インストールなし）


In [ ]:
!pip install torch-geometric 
# torch-scatter torch-sparse torch-cluster


In [ ]:
import torch

torch_ver = torch.__version__.split("+")[0]          # 例: "2.1.0+cu121" -> "2.1.0"
cuda_ver  = torch.version.cuda                      # 例: "12.1" / CPUなら None
cuda_tag  = "cpu" if cuda_ver is None else "cu" + cuda_ver.replace(".", "")

url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print("PyG wheels:", url)

# 必要なやつだけ入れる（radius_graphなら torch_cluster が本体）
!pip -q install torch-cluster -f {url}

# もし他も使うならまとめて
# !pip -q install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f {url}
# !pip -q install torch_geometric


In [ ]:
from copy import deepcopy
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

# 推論で使うデータセット
dataset_root = '/kaggle/input/dam-break-left'

# 試したい dt を列挙
DT_LIST = [0.006, 0.003]  # ここを編集

# 出力先
output_root = REPO / 'rollouts_dt'

# 追加でモデルを置いたデータセットがある場合は指定 (例: 'gns-models')
MODEL_DATASET_SLUG = "gns-40k-100-20/pytorch/default/1"

with cfg_path.open('r', encoding='utf-8') as f:
    base_cfg = yaml.safe_load(f)

base_cfg['method'] = 'gns'
base_cfg['rollout_inference_max_examples'] = 1
base_cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

# 既にモデルが置かれている場所があれば自動で使う
model_candidates = []
if MODEL_DATASET_SLUG: Path(f'/kaggle/input/{MODEL_DATASET_SLUG}')
model_candidates.extend([
    Path('/kaggle/input/gns-codes') / 'models',
    REPO / 'models',
])
for candidate in model_candidates:
    if candidate.exists():
        base_cfg['model_path'] = str(candidate)
        break

def _dt_label(value: float) -> str:
    return str(value).replace('.', 'p')

cfg_variants = []
for dt in DT_LIST:
    cfg_dt = deepcopy(base_cfg)
    method_opts = cfg_dt.setdefault('method_options', {}).setdefault('hamiltonian_sph', {})
    integrator = method_opts.setdefault('integrator', {})
    integrator['dt'] = float(dt)
    integrator['dt_source'] = 'config'
    label = _dt_label(float(dt))
    cfg_dt['output_path'] = str(output_root / f"dt_{label}")
    cfg_dt['output_filename'] = f"rollout_dt_{label}"
    cfg_variants.append((label, cfg_dt))


In [ ]:
%cd /kaggle/working/code

import subprocess
import yaml
from pathlib import Path

cfg_path = Path('config_rollout.yaml')
for label, cfg_dt in cfg_variants:
    with cfg_path.open('w', encoding='utf-8') as f:
        yaml.safe_dump(cfg_dt, f, allow_unicode=True)
    cmd = ["python", "src/train.py", "--config", "config_rollout.yaml"]
    print(f"[dt={label}] {' '.join(cmd)}")
    subprocess.run(cmd, check=True)
    subprocess.run(["python", "analyze_rollouts.py", "--rollouts-dir", cfg_dt['output_path']], check=True)
